In [1]:
import time
import cv2
import numpy as np
import chainer
import glob
import os
from functools import partial
from matplotlib import pyplot as plt
from chainer import serializers, optimizers, Variable, iterators,training, cuda
from chainer.datasets import TransformDataset
from chainer.training import extensions
from chainer import Reporter, report, report_scope
import chainer.functions as F
from yolov2 import *
from lib.utils import *
from lib.preprocess import _offset_boxes, clip_boxes
#from lib.image_generator import *
from lib.data_generator import *
from darknet19 import *


/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def copy_conv_layer(src, dst, layers):
    for i in layers:
        src_layer = eval("src.conv%d" % i)
        dst_layer = eval("dst.conv%d" % i)
        dst_layer.W = src_layer.W
        dst_layer.b = src_layer.b

def copy_bias_layer(src, dst, layers):
    for i in layers:
        src_layer = eval("src.bias%d" % i)
        dst_layer = eval("dst.bias%d" % i)
        dst_layer.b = src_layer.b

def copy_bn_layer(src, dst, layers):
    for i in layers:
        src_layer = eval("src.bn%d" % i)
        dst_layer = eval("dst.bn%d" % i)
        dst_layer.N = src_layer.N
        dst_layer.avg_var = src_layer.avg_var
        dst_layer.avg_mean = src_layer.avg_mean
        dst_layer.gamma = src_layer.gamma
        dst_layer.eps = src_layer.eps


In [3]:

def convert(batch, device):

    #for pair in batch:
        #print(pair[1])
    return chainer.dataset.convert.concat_examples(batch, device, padding=0)

def print_obs(t):
    print("trainer.observation", trainer.observation)
    print("updater.loss", updater.loss_func)
    
def save_model(t):
    
    serializers.save_hdf5("%s/yolov2_load.model" % ("./backup/result"), model)



In [4]:
# hyper parameters
#train_sizes = [320, 352, 384, 416, 448]
#train_sizes = [448]
initial_weight_file = "./backup/result/yolov2_load.model"
#initial_weight_file = None
trained_weight_file = "./darknet19_448.model"
snapshot=None
#snapshot = "./backup/result/snapshot_iter_4968"
backup_path = "backup"
backup_file = "%s/backup.model" % (backup_path)
batch_size =6
epoch = 1000
max_batches = 20000
learning_rate = 1e-5
learning_schedules = { 
    "0"    : 1e-5,
    "500"  : 1e-4,
    "10000": 1e-5,
    "20000": 1e-6 
}

lr_decay_power = 4
momentum = 0.9
weight_decay = 0.005
n_classes = 3
n_boxes = 5
partial_layer = 18

start = time.time()
imageNet_data = ImageNet_data("/home/ubuntu/sdcard/YOLOv2-master/XmlToTxt/water_bottle_img",
"/home/ubuntu/sdcard/YOLOv2-master/XmlToTxt/water_bottle_bbox", "/home/ubuntu/sdcard/YOLOv2-master/XmlToTxt/images_list.txt", n_classes )

In [5]:
trained_model = Darknet19()
serializers.load_npz(trained_weight_file, trained_model) # load saved model
trained_model = Darknet19Predictor(trained_model)
if initial_weight_file is None:
    print("inititializing with the darknet19_448 weights")
    yolov2 = YOLOv2(n_classes=n_classes, n_boxes=n_boxes)
    copy_conv_layer(trained_model.predictor, yolov2, range(1, partial_layer+1))
    copy_bias_layer(trained_model.predictor, yolov2, range(1, partial_layer+1))
    copy_bn_layer(trained_model.predictor, yolov2, range(1, partial_layer+1))
    model = YOLOv2Predictor(yolov2)
elif not (snapshot is None) :
    yolov2 = YOLOv2(n_classes=n_classes, n_boxes=n_boxes)
    model = YOLOv2Predictor(yolov2)
    serializers.load_npz(snapshot, model)

else :
    print("initializing with serial_load backup weight")
    yolov2 = YOLOv2(n_classes=n_classes, n_boxes=n_boxes)
    model = YOLOv2Predictor(yolov2)
    serializers.load_hdf5(initial_weight_file, model)



initializing with serial_load backup weight


In [6]:
model.predictor.train = True
model.predictor.finetune = False  ####or True ??
cuda.get_device(0).use()
model.to_gpu()

optimizer = optimizers.MomentumSGD(lr=learning_rate, momentum=momentum)
optimizer.use_cleargrads()
optimizer.setup(model)
model.predictor.conv1.disable_update()
model.predictor.conv2.disable_update()
model.predictor.conv3.disable_update()
model.predictor.conv4.disable_update()
model.predictor.conv5.disable_update()
model.predictor.conv6.disable_update()
model.predictor.conv7.disable_update()
model.predictor.conv8.disable_update()
model.predictor.conv9.disable_update()
model.predictor.conv10.disable_update()
model.predictor.conv11.disable_update()
model.predictor.conv12.disable_update()
model.predictor.conv13.disable_update()
model.predictor.conv14.disable_update()
model.predictor.conv15.disable_update()
model.predictor.conv16.disable_update()
#model.predictor.conv17.disable_update()
#model.predictor.conv18.disable_update()
#model.predictor.conv19.disable_update()
optimizer.add_hook(chainer.optimizer.WeightDecay(weight_decay))


In [7]:
def transform(inputs, train=True):
    #we take only one input
    img, label= inputs
    boxes = label.copy()
    im = img.copy()
    im = np.asarray(im, dtype=np.float32) /255.0
    im = np.transpose(im, (1, 2, 0))
    shape= im.shape
    #random_angle = np.random.uniform(0, 360)
    im, trans_param = imcv2_affine_trans(im)
    scale, offs, flip = trans_param
    print("label", label, "scale, offs, flip", scale, offs, flip, "shape init" , shape)
    boxes = _offset_boxes(boxes, im.shape, scale, offs, flip)
    sample_image = im.copy()
    boxes = np.asarray(boxes, dtype=np.float32)
    print("box", boxes)
    box = Box(boxes[0][1]*shape(1), boxes[0][2]*shape(0), boxes[0][3]*shape(1),boxes[0][4]*shape(0))
    cv2.rectangle(
               sample_image,
               box.int_left_top(), box.int_right_bottom(),
               (255, 0, 255),
               3
           )
    cv2.imshow("image", sample_image)

    cv2.waitKey(0)
    cv2.destroyAllWindows()
    print("label", boxes)
    im = cv2.resize(im, shape)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    im = imcv2_recolor(im)
    return img, label

    

In [8]:
reporter = chainer.Reporter()


#Create Train and test datset
train, test = imageNet_data.train_val_test()

#Transformed dataset, first creating function
train_transf1 = partial(transform)
print("we transform data set for train")
train_trans1 = TransformDataset(train, train_transf1)

train_iter = iterators.SerialIterator(train, batch_size)
test_iter = iterators.SerialIterator(test, batch_size, repeat=False, shuffle=True)
test=1
val_interval = (1 if test else 100000), 'epoch'
log_interval = (30 if test else 1000), 'iteration'



with chainer.using_config('debug', True):
    # Set up a trainer
    updater = training.StandardUpdater(train_iter, optimizer, loss_func=model, converter=convert, device=0)
    trainer = training.Trainer(updater, (100, 'epoch'), out="./backup/result")
updater.connect_trainer(trainer)
# Evaluate the model with the test dataset for each epoch
trainer.extend(extensions.Evaluator(test_iter, model, converter=convert, device=0), trigger=val_interval)

# Dump a computational graph from 'loss' variable at the first iteration
# The "main" refers to the target link of the "main" optimizer.
trainer.extend(extensions.dump_graph('main/loss'))

# Take a snapshot at each epoch
#trainer.extend(extensions.snapshot(), trigger=(args.epoch, 'epoch'))
trainer.extend(extensions.snapshot(), trigger=(20, 'epoch'))
trainer.extend(save_model,  trigger=(10, 'epoch'))
# Write a log of evaluation statistics for each epoch
trainer.extend(extensions.LogReport(trigger=log_interval))
trainer.extend(extensions.PrintReport(
      ['epoch', 'main/images', 'main/loss', 'validation/main/loss', 'elapsed_time']), trigger=log_interval)

# Print selected entries of the log to stdout
# Here "main" refers to the target link of the "main" optimizer again, and
# "validation" refers to the default name of the Evaluator extension.
# Entries other than 'epoch' are reported by the Classifier link, called by
# either the updater or the evaluator.

# Plot graph for loss for each epoch
if extensions.PlotReport.available():
    trainer.extend(extensions.PlotReport(
        ['main/loss', 'validation/main/loss'],
        x_key='epoch', file_name='loss.png'))
else:
    print('Warning: PlotReport is not available in your environment')
# Print a progress bar to stdout
trainer.extend(extensions.ProgressBar(update_interval=500))




we transform data set for train


In [ ]:
with chainer.using_config('debug', True):
    print("start")
    # Run the training
    trainer.run()

start
epoch       main/images  main/loss   validation/main/loss  elapsed_time
0           93           25.9875                           286.419       
0           273          17.3471                           531.411       


In [ ]:
serializers.save_hdf5("%s/yolov2_final.model" % ("./backup/result"), model)
